# Data Loading Pipeline

This notebook loads and transforms Understat data for the **2023/24 Premier League season**.

**Data Sources:**
- Understat CSV files (match xG data, player rosters)

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Configuration
SEASON = 2023  # Latest available season in Understat data (2023/24 season)
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# Understat CSV paths
UNDERSTAT_TEAM_CSV = "./understat_matches_team.csv"
UNDERSTAT_ROSTER_CSV = "./understat_matches_roster.csv"

print("="*60)
print("PREMIER LEAGUE DATA LOADING PIPELINE")
print("="*60)
print(f"\n📅 Target Season: 2023/24")
print(f"📁 Data Directory: {DATA_DIR}")
print(f"📊 Data Source: Understat CSV files")

PREMIER LEAGUE DATA LOADING PIPELINE

📅 Target Season: 2023/24
📁 Data Directory: ./data
📊 Data Source: Understat CSV files


## 1. Load Understat Match Data
Load match-level team and player data from Understat CSV files for the 2023/24 season.

In [2]:
print("\n" + "="*60)
print("LOADING UNDERSTAT DATA")
print("="*60)

# Load Understat match data
print("\n📊 Loading Understat match data...")
understat_matches = pd.read_csv(UNDERSTAT_TEAM_CSV)
understat_matches['date'] = pd.to_datetime(understat_matches['date'])

# Filter to 2023 season (2023/24 in Understat is year 2023)
understat_matches_2023 = understat_matches[understat_matches['season'] == SEASON].copy()

print(f"✓ Understat matches loaded: {len(understat_matches_2023)} matches")
print(f"  Date range: {understat_matches_2023['date'].min()} to {understat_matches_2023['date'].max()}")

# Load player roster data
print("\n👤 Loading Understat player data...")
understat_players = pd.read_csv(UNDERSTAT_ROSTER_CSV)

# Extract match IDs and dates from team matches
match_dates = understat_matches[['id', 'date', 'season']].copy()
match_dates.columns = ['match_id', 'date', 'season']
match_dates['date'] = pd.to_datetime(match_dates['date'])

# Join with player data
understat_players['match_id'] = understat_players['match_link'].str.extract(r'/match/(\d+)').astype(int)
understat_players = understat_players.merge(match_dates, on='match_id', how='left')

# Filter to 2023 season
understat_players_2023 = understat_players[understat_players['season'] == SEASON].copy()

print(f"✓ Understat player data loaded: {len(understat_players_2023)} player-match records")

print("\n✅ Understat data loaded successfully!")


LOADING UNDERSTAT DATA

📊 Loading Understat match data...
✓ Understat matches loaded: 380 matches
  Date range: 2023-08-11 19:00:00 to 2024-05-19 15:00:00

👤 Loading Understat player data...
✓ Understat player data loaded: 11384 player-match records

✅ Understat data loaded successfully!


## 2. Transform Match Data to Team Perspective
Convert wide-format match data to team-match format (one row per team per match).

In [3]:
print("\n" + "="*60)
print("TRANSFORMING MATCH DATA")
print("="*60)

def transform_match_to_team_perspective(df):
    """Transform match data from wide to long format (team perspective)."""
    # Home team perspective
    home_df = pd.DataFrame({
        'match_id': df['id'],
        'date': pd.to_datetime(df['date']),
        'season': df['season'],
        'league': df['league'],
        'team_name': df['team_h'],
        'opponent': df['team_a'],
        'venue': 'Home',
        'goals_for': df['h_goals'],
        'goals_against': df['a_goals'],
        'xG': df['h_xg'],
        'xGA': df['a_xg'],
        'shots': df['h_shot'],
        'shots_against': df['a_shot'],
        'shots_on_target': df['h_shotOnTarget'],
        'shots_on_target_against': df['a_shotOnTarget'],
        'deep': df['h_deep'],
        'deep_allowed': df['a_deep'],
        'ppda': df['h_ppda'],
        'ppda_allowed': df['a_ppda'],
    })
    
    # Away team perspective
    away_df = pd.DataFrame({
        'match_id': df['id'],
        'date': pd.to_datetime(df['date']),
        'season': df['season'],
        'league': df['league'],
        'team_name': df['team_a'],
        'opponent': df['team_h'],
        'venue': 'Away',
        'goals_for': df['a_goals'],
        'goals_against': df['h_goals'],
        'xG': df['a_xg'],
        'xGA': df['h_xg'],
        'shots': df['a_shot'],
        'shots_against': df['h_shot'],
        'shots_on_target': df['a_shotOnTarget'],
        'shots_on_target_against': df['h_shotOnTarget'],
        'deep': df['a_deep'],
        'deep_allowed': df['h_deep'],
        'ppda': df['a_ppda'],
        'ppda_allowed': df['h_ppda'],
    })
    
    # Combine
    team_matches = pd.concat([home_df, away_df], ignore_index=True)
    team_matches = team_matches.sort_values(['date', 'match_id']).reset_index(drop=True)
    
    # Add derived columns
    team_matches['goal_diff'] = team_matches['goals_for'] - team_matches['goals_against']
    team_matches['result'] = team_matches.apply(
        lambda row: 'W' if row['goals_for'] > row['goals_against']
        else ('D' if row['goals_for'] == row['goals_against'] else 'L'),
        axis=1
    )
    team_matches['points'] = team_matches['result'].map({'W': 3, 'D': 1, 'L': 0})
    
    return team_matches

# Transform 2023/24 season data
print("\n🔄 Transforming to team-match format...")
team_matches_2023 = transform_match_to_team_perspective(understat_matches_2023)

print(f"✓ Team matches created: {len(team_matches_2023)} records")
print(f"  Unique teams: {team_matches_2023['team_name'].nunique()}")
if team_matches_2023['team_name'].nunique() > 0:
    print(f"  Matches per team: {len(team_matches_2023) / team_matches_2023['team_name'].nunique():.1f}")

# Save
team_matches_2023.to_csv(os.path.join(DATA_DIR, "team_matches_2023.csv"), index=False)
print(f"\n💾 Saved: team_matches_2023.csv")


TRANSFORMING MATCH DATA

🔄 Transforming to team-match format...
✓ Team matches created: 760 records
  Unique teams: 20
  Matches per team: 38.0

💾 Saved: team_matches_2023.csv


## 3. Process Player Data
Transform player roster data with match information.

In [4]:
print("\n" + "="*60)
print("PROCESSING PLAYER DATA")
print("="*60)

# Get team names from match data
match_teams = understat_matches_2023[['id', 'team_h', 'team_a']].copy()

# Process player data
players_2023 = understat_players_2023.copy()

# Add team names based on h_a indicator
players_2023 = players_2023.merge(
    match_teams,
    left_on='match_id',
    right_on='id',
    how='left',
    suffixes=('', '_match')
)

players_2023['team_name'] = players_2023.apply(
    lambda row: row['team_h'] if row['h_a'] == 'h' else row['team_a'],
    axis=1
)

players_2023['opponent'] = players_2023.apply(
    lambda row: row['team_a'] if row['h_a'] == 'h' else row['team_h'],
    axis=1
)

# Rename columns
players_2023 = players_2023.rename(columns={
    'time': 'minutes',
    'player': 'player_name',
})

# Select relevant columns
player_cols = [
    'match_id', 'date', 'season', 'player_id', 'player_name',
    'team_name', 'opponent', 'h_a', 'position', 'minutes',
    'goals', 'assists', 'xG', 'xA', 'shots', 'key_passes',
    'xGChain', 'xGBuildup', 'yellow_card', 'red_card'
]

players_2023 = players_2023[player_cols].sort_values(['date', 'match_id'])

print(f"✓ Player data processed: {len(players_2023)} player-match records")
print(f"  Unique players: {players_2023['player_name'].nunique()}")

# Save
players_2023.to_csv(os.path.join(DATA_DIR, "player_matches_2023.csv"), index=False)
print(f"\n💾 Saved: player_matches_2023.csv")


PROCESSING PLAYER DATA
✓ Player data processed: 11384 player-match records
  Unique players: 569

💾 Saved: player_matches_2023.csv


## 4. Load Historical Player Data (for context)
Load player stats from previous seasons (2015-2023) for comparison and profiling.

In [5]:
print("\n" + "="*60)
print("LOADING HISTORICAL PLAYER DATA")
print("="*60)

# Load all historical Understat player data (for player profiling context)
print("\n📚 Loading historical player data...")

# Get all seasons before 2023
historical_players = understat_players[understat_players['season'] < 2023].copy()

print(f"✓ Historical player data loaded: {len(historical_players)} records")
print(f"  Seasons: {sorted(historical_players['season'].unique())}")
print(f"  Unique players: {historical_players['player'].nunique()}")

# Save for reference
historical_players.to_csv(os.path.join(DATA_DIR, "player_matches_historical.csv"), index=False)
print(f"\n💾 Saved: player_matches_historical.csv")


LOADING HISTORICAL PLAYER DATA

📚 Loading historical player data...
✓ Historical player data loaded: 84707 records
  Seasons: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
  Unique players: 1593

💾 Saved: player_matches_historical.csv


## 5. Data Summary
Display summary statistics of loaded data.

In [6]:
print("\n" + "="*60)
print("DATA LOADING SUMMARY")
print("="*60)

print("\n📊 2023/24 SEASON DATA:")
print(f"  Team Matches: {len(team_matches_2023):,}")
print(f"  Player Matches: {len(players_2023):,}")
print(f"  Unique Teams: {team_matches_2023['team_name'].nunique()}")
print(f"  Unique Players: {players_2023['player_name'].nunique()}")
print(f"  Date Range: {team_matches_2023['date'].min().date()} to {team_matches_2023['date'].max().date()}")

print("\n📚 HISTORICAL DATA (for context):")
print(f"  Player Matches: {len(historical_players):,}")
print(f"  Seasons: {len(historical_players['season'].unique())}")
print(f"  Date Range: {historical_players['date'].min().date()} to {historical_players['date'].max().date()}")

print("\n" + "="*60)
print("✅ DATA LOADING COMPLETE!")
print("="*60)

print("\n📁 Files saved in ./data/:")
print("  - team_matches_2023.csv")
print("  - player_matches_2023.csv")
print("  - player_matches_historical.csv")

print("\n📊 Data Coverage:")
print("  ✓ Match-level xG, xGA, shots, PPDA")
print("  ✓ Player-level goals, assists, xG, xA")
print("  ✓ Team tactical metrics (pressing, deep completions)")


DATA LOADING SUMMARY

📊 2023/24 SEASON DATA:
  Team Matches: 760
  Player Matches: 11,384
  Unique Teams: 20
  Unique Players: 569
  Date Range: 2023-08-11 to 2024-05-19

📚 HISTORICAL DATA (for context):
  Player Matches: 84,707
  Seasons: 8
  Date Range: 2015-08-08 to 2023-05-28

✅ DATA LOADING COMPLETE!

📁 Files saved in ./data/:
  - team_matches_2023.csv
  - player_matches_2023.csv
  - player_matches_historical.csv

📊 Data Coverage:
  ✓ Match-level xG, xGA, shots, PPDA
  ✓ Player-level goals, assists, xG, xA
  ✓ Team tactical metrics (pressing, deep completions)


## Next Steps

Run the feature engineering notebook to process this data further: **`02_feature_engineering.ipynb`**

### 📊 Understat Data Coverage

The Understat dataset provides comprehensive metrics for tactical and scouting analysis:

**Team-Level:**
- xG (Expected Goals) and xGA (Expected Goals Against)
- Shots, shots on target, shot accuracy
- PPDA (Passes Allowed Per Defensive Action) - pressing intensity
- Deep completions - attacking penetration
- Goals, results, points

**Player-Level:**
- Goals, assists, minutes played
- xG, xA (Expected Assists)
- Shots, key passes
- xGChain, xGBuildup - involvement in attack
- Position, team, opponent context

This data is **sufficient for all project objectives**: tactical clustering, player scouting, opponent analysis, and performance trends.

**Note**: Using 2023/24 season data (latest available in CSV exports).